In [1]:
# imports needed packages
from bs4 import BeautifulSoup as bs 
import numpy as np
import pandas as pd
import requests
import re
import xlrd 
import openpyxl

## Population Census DataFrame

In [42]:
#checks site status
url = 'https://www.census.gov/data/tables/time-series/dec/popchange-data-text.html'

req = requests.get(url)

print(req.status_code)


200


In [43]:
soup = bs(req.content, 'html.parser')

In [63]:
def convert(element):
    return element.string if element.string else None

In [64]:
state_names = list(map(convert, soup.select('th')))

In [106]:
#state_names

In [66]:
def remove_until_element(lst, element):
    try:
        index = lst.index(element)
        return lst[index:]
    except ValueError:
        return lst
    
state_names = remove_until_element(state_names, 'United States')

In [67]:
residents = list(map(convert, soup.select('td+ td , #POP_US+ td')))

In [68]:
# Creates two seperate arrays for the resident population and percent change
resident_population = []
percent_change = []

for element in residents:
    
    if '%' in element:
        percent_change.append(element)
    
    else:
        resident_population.append(element)


In [69]:
num_cols = 12
num_rows = 57
reshaped_list = np.array(resident_population).reshape(num_rows, num_cols)

census_resident_pop = pd.DataFrame(reshaped_list, columns=[f'{i}' for i in range(1, num_cols + 1)])

In [70]:
start_year = 2020

for i in range(1, 13):

    original_column_name = str(i)

    census_resident_pop.rename(columns={original_column_name: f'{start_year} Resident Population'}, inplace=True)

    start_year -=10


In [71]:
num_cols = 12
num_rows = 57
reshaped_list = np.array(percent_change).reshape(num_rows, num_cols)

census_percent_change = pd.DataFrame(reshaped_list, columns=[f'{i}' for i in range(1, num_cols + 1)])

start_year = 2020

for i in range(1, 13):

    original_column_name = str(i)

    census_percent_change.rename(columns={original_column_name: f'{start_year} Percentage Change'}, inplace=True)

    start_year -=10



In [72]:
census_df = pd.DataFrame(data=state_names, columns=['State Names'])

In [105]:
#census_df

In [74]:
census_df = census_df.join(census_resident_pop)

**Note:** Due to reasons of data encoding and model construction, the percentage will not be included in the dataframe. In addition to this, certain rows of the dataframe such as the District of Columbia and Puerto Rico will not be included in the dataframe. Puerto Rico for its lack of statehood, and DC due to it already being encompased in a state.

In [76]:
census_df.drop([0, 1, 2, 3, 4, 13, 56], inplace=True)

In [77]:
census_df = census_df.reset_index(drop=True)

In [104]:
#census_df.head()

In [79]:
census_df.to_csv('United_States_Census_Data.csv')

## Median Income DataFrame

In [91]:
url = 'https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=1984-01-01#'

req = requests.get(url)

print(req.status_code)

200


In [92]:
soup = bs(req.content, 'html.parser')

In [93]:
state_names_income = list(map(convert, soup.select('th span')))

In [94]:
state_income_df = pd.DataFrame(data=state_names_income, columns=['State Names'])

state_income_df.drop([0], inplace=True)

state_income_df = state_income_df.reset_index(drop=True)

In [96]:
beginning_year = 1984

for i in range(38):
    url = f'https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od={beginning_year}-01-01#'

    req = requests.get(url)

    soup = bs(req.content, 'html.parser')

    median_request = list(map(convert, soup.select('.fred-rls-elm-nm-td+ .fred-rls-elm-vl-td')))

    state_income_df['{} Median Income'.format(beginning_year)] = median_request
    
    beginning_year +=1

In [98]:
def clean_and_convert(string):
    cleaned_string = re.sub(r'[^\d]', '', string)
    
    return int(cleaned_string)

In [99]:
beginning_year = 1984

for i in range(38):
    state_income_df['{} Median Income'.format(beginning_year)] = state_income_df['{} Median Income'.format(beginning_year)].map(clean_and_convert)

    beginning_year +=1

In [101]:
state_income_df.drop([0, 9], inplace=True)

state_income_df = state_income_df.reset_index(drop=True)

In [103]:
#state_income_df.head(20)

In [107]:
state_income_df.to_csv('United_States_Median_Income_by_State.csv')

## Unemployment Rate DataFrame

In [126]:
unemployment_df = pd.read_excel('/Users/emmanuely/Downloads/Unemployment.xlsx')

In [127]:
unemployment_df.head(6)

,"Unemployment and median household income for the U.S., States, and counties, 2000–22",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99
0,Sources: Unemployment: U.S. Department of Labo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"For definitions of rural classifications, see ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"This table was prepared by USDA, Economic Rese...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FIPS_Code,State,Area_Name,Rural_Urban_Continuum_Code_2013,Urban_Influence_Code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Civilian_labor_force_2022,Employed_2022,Unemployed_2022,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021
4,00000,US,United States,NaN,NaN,NaN,142601576,136904853,5696723,4,...,162229903,153544980,8684923,5.4,164781642,158766998,6014644,3.7,69717,NaN
5,01000,AL,Alabama,NaN,NaN,NaN,2147173,2047731,99442,4.6,...,2259349,2183330,76019,3.4,2286028,2226670,59358,2.6,53990,100


In [128]:
unemployment_df.drop([0, 1, 2], inplace=True)

unemployment_df = unemployment_df.reset_index(drop=True)

In [129]:
unemployment_df.columns = unemployment_df.iloc[0]

unemployment_df = unemployment_df.iloc[1:].reset_index(drop=True)

In [130]:
unemployment_df.drop([0], inplace=True)

unemployment_df = unemployment_df.reset_index(drop=True)

In [131]:
unemployment_df.rename(columns={'State': 'State Names'})

,FIPS_Code,State Names,Area_Name,Rural_Urban_Continuum_Code_2013,Urban_Influence_Code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Civilian_labor_force_2022,Employed_2022,Unemployed_2022,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021
0,01000,AL,Alabama,NaN,NaN,NaN,2147173,2047731,99442,4.6,...,2259349,2183330,76019,3.4,2286028,2226670,59358,2.6,53990,100
1,01001,AL,"Autauga County, AL",2,2,1,21861,20971,890,4.1,...,26545,25809,736,2.8,26789,26181,608,2.3,66444,123.1
2,01003,AL,"Baldwin County, AL",3,2,1,69979,67370,2609,3.7,...,99953,97034,2919,2.9,102849,100432,2417,2.4,65658,121.6
3,01005,AL,"Barbour County, AL",6,6,0,11449,10812,637,5.6,...,8280,7821,459,5.5,8241,7906,335,4.1,38649,71.6
4,01007,AL,"Bibb County, AL",1,1,1,8623,8160,463,5.4,...,8641,8347,294,3.4,8726,8507,219,2.5,48454,89.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,72145,PR,"Vega Baja Municipio, PR",1,1,1,19805,17607,2198,11.1,...,14172,12748,1424,10,14232,13180,1052,7.4,NaN,NaN
3272,72147,PR,"Vieques Municipio, PR",7,12,0,2441,2120,321,13.2,...,2804,2469,335,11.9,2742,2554,188,6.9,NaN,NaN
3273,72149,PR,"Villalba Municipio, PR",2,2,1,7636,6648,988,12.9,...,7697,6796,901,11.7,7790,6982,808,10.4,NaN,NaN
3274,72151,PR,"Yabucoa Municipio, PR",1,1,1,10613,9005,1608,15.2,...,9156,8216,940,10.3,9278,8504,774,8.3,NaN,NaN


In [132]:
def drop_rows_with_comma(df):
    indices_to_drop = df[df['Area_Name'].str.contains(',')].index
    
    df.drop(indices_to_drop, inplace=True)
    
    return df

unemployment_df = drop_rows_with_comma(unemployment_df)

In [133]:
unemployment_df.rename(columns={'Area_Name': 'State Names'}, inplace=True)

unemployment_df.drop(columns={'FIPS_Code', 'State'}, axis=1, inplace=True)

**Note:** *Just something to note since this hasn't really been done for the other datasets. The columns I about to delete bear no impact on the calculations I am attempting to make for my predictions. In addition, many of these columns have overwhelming amounts of NAN values that would need to be imputed, which would introduce a massive amount of bias into our model. As such, I find that the best option is to delete these columns*

In [134]:
unemployment_df.drop(columns={'Rural_Urban_Continuum_Code_2013', 'Urban_Influence_Code_2013', 'Metro_2013',
                              'Median_Household_Income_2021', 'Med_HH_Income_Percent_of_State_Total_2021'}, axis=1, inplace=True)

In [135]:
unemployment_df.drop([3197], inplace=True)

In [137]:
start_year = 2000

for i in unemployment_df.columns.to_list():
    if start_year != 2023:
        
        unemployment_df.drop(columns={f'Civilian_labor_force_{start_year}', f'Employed_{start_year}', 
                                      f'Unemployment_rate_{start_year}'}, axis=1, inplace=True)
    
        start_year +=1

In [138]:
unemployment_df = unemployment_df.reset_index(drop=True)

In [139]:
# District of Columbia was inluded twice. We don't even want it in the first place
unemployment_df.drop([8, 9], inplace=True)

unemployment_df = unemployment_df.reset_index(drop=True)

In [141]:
# unemployment_df.head()

In [142]:
unemployment_df.to_csv('United_States_Unemployment_Data.csv')

## Fertility DataFrame

In [146]:
fertility_df = pd.read_csv('/Users/emmanuely/Downloads/data-table.csv')

In [147]:
fertility_df.drop(columns=['URL', 'FERTILITY RATE'], axis=1, inplace=True)

In [148]:
fertility_df

,YEAR,STATE,BIRTHS
0,2021,AL,58054
1,2021,AK,9367
2,2021,AZ,77916
3,2021,AR,35965
4,2021,CA,420608
...,...,...,...
445,2005,VA,104555
446,2005,WA,82703
447,2005,WV,20836
448,2005,WI,70984


In [149]:
state_abbr = {
        'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
        'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
        'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
        'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
        'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
        'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
        'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
        'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
        'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
        'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
    }

def switch_state_names(state):
    if state in state_abbr:
        state = state_abbr[state]
        return
    

fertility_df['STATE'] = fertility_df['STATE'].map(state_abbr)


In [150]:
fertility_df = fertility_df.reindex(columns=['STATE', 'YEAR', 'BIRTHS'])

In [151]:
fertility_df = fertility_df.pivot(index='STATE', columns='YEAR', values='BIRTHS').reset_index()

In [152]:
new_columns = {'YEAR':None, 'STATE':'State Names', 2005: '2005 BIRTHS', 2014: '2014 BIRTHS', 
               2015: '2015 BIRTHS', 2016: '2016 BIRTHS', 2017: '2017 BIRTHS', 
               2018: '2018 BIRTHS', 2019: '2019 BIRTHS', 2020: '2020 BIRTHS', 
               2021: '2021 BIRTHS'}

In [153]:
fertility_df.rename(columns=new_columns, inplace=True)

In [155]:
#The isolated year of 2005 can drastically affect our time-series model, hence my decision to remove it
fertility_df.drop(columns=['2005 BIRTHS'], axis=1, inplace=True)

In [157]:
# fertility_df.head()

In [158]:
fertility_df.to_csv('United_States_Fertility_Rates.csv')